In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
import open3d as o3d
from torch.utils.data import Dataset, DataLoader, Subset
from dataclasses import dataclass
import sys
from pathlib import Path
from pytorch3d.loss import chamfer_distance
from tqdm import tqdm

sys.path.append(str(Path.cwd().parent))

import AutoEncoder.autoencoders as ae
from AutoEncoder.encoder import PointCloudAutoEncoder
from Helpers.data import PointCloudDataset
import Helpers.PointCloudOpen3d as pc
from Helpers.nsdf_live_sampler import sample_sdf
from SignDistanceModel.sdf_models import SDFRegressionModel

if torch.cuda.is_available():
    device = "cuda"

elif hasattr(torch.backends, "mps") and torch.backends.mps.is_available():
    device = "mps"
else:
    device = "cpu"

print(f'Using: {device}')


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
Failed to sample sdf on file:  ./Data/vdbs/airplane/train/airplane_0001.vdb
Using: cuda


In [2]:
train_dataset_1024 = PointCloudDataset("../Data/ModelNet40", 1024, 'train', object_classes =  None )
test_dataset_1024 = PointCloudDataset("../Data/ModelNet40", 1024, 'test', object_classes =  None)

test_size = len(test_dataset_1024)
split_idx = test_size // 2
indices = list(range(test_size))
val_dataset_1024 = Subset(test_dataset_1024, indices[:split_idx])
test_dataset_1024 = Subset(test_dataset_1024, indices[split_idx:])

train_loader_1024= DataLoader(train_dataset_1024, batch_size = 64, shuffle = True)
val_loader_1024 = DataLoader(val_dataset_1024, batch_size = 128, shuffle = False)
test_loader_1024 = DataLoader(test_dataset_1024, batch_size = 128, shuffle = False)

/home/amuseth/dev/src/Neural-Sign-Distance-Learning/Helpers/data.py:69: RuntimeWarning: invalid value encountered in divide
  scaled_points = ((points - min_coords) / range_coords) #scales points on range [0, 1]
/home/amuseth/dev/src/Neural-Sign-Distance-Learning/Helpers/data.py:70: RuntimeWarning: invalid value encountered in cast
  scaled_points = (scaled_points * (2**num_bits - 1)).astype(int) # scales points to [0, 2^num_bits]


In [3]:
# Load model
model = ae.ConvAE_6800T(1024, 256)
model.load_state_dict(torch.load('../Data/trained_autoencoders/1024_256/Conv_6800T', weights_only=True))
print(model.to('cpu'))
model = model.to('cpu')

ConvAE_6800T(
  (encoder): ConvEncoder_6800T(
    (conv1): Conv1d(3, 32, kernel_size=(1,), stride=(1,))
    (conv2): Conv1d(32, 32, kernel_size=(9,), stride=(1,), padding=(4,))
    (conv3): Conv1d(32, 32, kernel_size=(9,), stride=(1,), padding=(4,))
    (conv4): Conv1d(32, 16, kernel_size=(8,), stride=(2,), padding=(3,))
    (conv5): Conv1d(16, 16, kernel_size=(8,), stride=(2,), padding=(3,))
    (conv6): Conv1d(16, 16, kernel_size=(8,), stride=(2,), padding=(3,))
    (lin1): Linear(in_features=2048, out_features=1024, bias=True)
    (lin2): Linear(in_features=1024, out_features=256, bias=True)
  )
  (decoder): ConvDecoder(
    (l1): Linear(in_features=256, out_features=1024, bias=True)
    (l2): Linear(in_features=1024, out_features=2048, bias=True)
    (l3): Linear(in_features=2048, out_features=3072, bias=True)
    (l4): Linear(in_features=3072, out_features=3072, bias=True)
  )
)


In [4]:
def visualize_random_reconstruction(model, x, device, transpose_out = False, show_org = True):
    '''
    Pick a random cloud from the dataset and show what it looks like before and after autoencoder reconstruction 
    First window is original point cloud 
    Second window is recontructed point cloud
    '''

    def show_cloud(x):
        pc.visualize_point_cloud(pc.get_point_cloud(x.cpu().numpy().astype(np.float64)))

    
    if show_org:
        show_cloud(x)

    with torch.no_grad():
        x = x.unsqueeze(0).permute(0,2,1).to(device)
        
        rec_x = model(x)[0].to('cpu')
        
        if transpose_out:
            rec_x = rec_x.T
        
        rec_x = rec_x

        show_cloud(rec_x)

#visualize_random_reconstruction(model, x, device)

In [ ]:
x = next(iter(val_loader_1024))
visualize_random_reconstruction(model, x['points'][0].to('cpu'), 'cpu')
print(x["filename"][0])

../Data/ModelNet40/sofa/test/sofa_0706.off


In [15]:
def train_one_mesh(filename, model, neural_encoding, optimizer, live_sampling=True, num_points=10000, num_epochs = 10000):
    train_losses = []
    val_losses = []
    criterion = nn.MSELoss()
    filename_split = filename.split('/')
    filename_split[2] = "vdbs"
    filename_split[-1]  = filename_split[-1][:-3] + "vdb" 
    vdb_filename = "".join([i + '/' for i in filename_split])[:-1]
    neural_encoding = neural_encoding.repeat(num_points, 1)
    for epoch in tqdm(range(num_epochs)):
        points, labels = sample_sdf(vdb_filename, num_points, device=device)
        perm = torch.randperm(num_points)
        points, labels = points[perm], labels[perm]
        optimizer.zero_grad()
        outputs = model(points, neural_encoding)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        if(epoch % 1000 == 0):
           print(labels[0], outputs[0])
           label_signs = torch.sign(labels)
           output_signs = torch.sign(outputs)
           correct_signs = (label_signs == output_signs).sum().item()
           print(correct_signs)
           print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

In [13]:
x = next(iter(val_loader_1024))

In [14]:
y = x["points"][0].unsqueeze(0).permute(0,2,1).to(device)
model.to(device) 
with torch.no_grad():
    neural_encoding = model.encoder(y)
sdf_model = SDFRegressionModel(3, 256, 128)
sdf_model.to(device)
optimizer = optim.AdamW(sdf_model.parameters(), lr=0.0001)
print(neural_encoding.shape)
train_one_mesh(x['filename'][0], sdf_model, neural_encoding, optimizer)

torch.Size([1, 256])


  0%|          | 1/10000 [00:00<22:11,  7.51it/s]

tensor([0.9580], device='cuda:0') tensor([-0.1284], device='cuda:0', grad_fn=<SelectBackward0>)
Epoch [1/10000], Loss: 0.3742


  1%|          | 103/10000 [00:08<13:08, 12.56it/s]

tensor([1.0816], device='cuda:0') tensor([0.1237], device='cuda:0', grad_fn=<SelectBackward0>)
Epoch [101/10000], Loss: 0.3168


...

 98%|█████████▊| 9803/10000 [13:47<00:18, 10.93it/s]

tensor([-0.5218], device='cuda:0') tensor([-0.1341], device='cuda:0', grad_fn=<SelectBackward0>)
Epoch [9801/10000], Loss: 0.1382


 99%|█████████▉| 9903/10000 [13:55<00:07, 12.19it/s]

tensor([0.8779], device='cuda:0') tensor([0.1265], device='cuda:0', grad_fn=<SelectBackward0>)
Epoch [9901/10000], Loss: 0.1379


100%|██████████| 10000/10000 [14:03<00:00, 11.85it/s]


In [16]:
train_one_mesh(x['filename'][0], sdf_model, neural_encoding, optimizer)

  0%|          | 2/10000 [00:00<14:38, 11.38it/s]

tensor([-0.3270], device='cuda:0') tensor([0.1707], device='cuda:0', grad_fn=<SelectBackward0>)
7891
Epoch [1/10000], Loss: 0.1373


 10%|█         | 1002/10000 [01:23<12:03, 12.44it/s]

tensor([-0.1702], device='cuda:0') tensor([0.2981], device='cuda:0', grad_fn=<SelectBackward0>)
7966
Epoch [1001/10000], Loss: 0.1340


 20%|██        | 2002/10000 [02:47<10:29, 12.71it/s]

...

 80%|████████  | 8002/10000 [11:13<02:41, 12.40it/s]

tensor([-0.5703], device='cuda:0') tensor([0.1685], device='cuda:0', grad_fn=<SelectBackward0>)
8190
Epoch [8001/10000], Loss: 0.1186


 90%|█████████ | 9002/10000 [12:39<01:20, 12.46it/s]

tensor([-0.4740], device='cuda:0') tensor([-0.3804], device='cuda:0', grad_fn=<SelectBackward0>)
8145
Epoch [9001/10000], Loss: 0.1172


100%|██████████| 10000/10000 [14:03<00:00, 11.86it/s]


In [18]:
y = x["points"][0].unsqueeze(0).permute(0,2,1).to(device)
model.to(device) 
with torch.no_grad():
    neural_encoding = model.encoder(y)
sdf_model2 = SDFRegressionModel(3, 256, 128)
sdf_model2.to(device)
optimizer = optim.AdamW(sdf_model2.parameters(), lr=0.001)
print(neural_encoding.shape)
train_one_mesh(x['filename'][0], sdf_model2, neural_encoding, optimizer, num_epochs = 100000)

torch.Size([1, 256])


  0%|          | 3/100000 [00:00<3:12:41,  8.65it/s]

tensor([0.9908], device='cuda:0') tensor([-0.0876], device='cuda:0', grad_fn=<SelectBackward0>)
4817
Epoch [1/100000], Loss: 0.3476


  0%|          | 23/100000 [00:02<2:59:42,  9.27it/s]


KeyboardInterrupt: 